In [1]:
source_file_path = None
destination_file_path = None

In [2]:
# Parameters
source_file_path = "/Users/nitinsharma/playground/finance/data/Acct Statement_XX8550_31052025 (1).xls"
destination_file_path = "/Users/nitinsharma/playground/finance/transformed/"


In [3]:
import pandas as pd
import os

In [4]:
df = pd.read_excel(source_file_path)

In [5]:
headless = df.drop(df.index[0:19])
headless = headless.reset_index(drop=True)

headless.columns = headless.iloc[0]      # Set new column names
headless = headless[1:]                  # Drop the row now used as header
headless = headless.reset_index(drop=True) 
headless = headless.drop(0)

In [6]:
mask = headless.map(lambda x: isinstance(x, str) and x.strip().strip('*') == '').all(axis=1)
cutoff_index = mask.idxmax()

tailless = headless.drop(headless.index[cutoff_index-2: ])

In [7]:
tailless.tail()

,Date,Narration,Chq./Ref.No.,Value Dt,Withdrawal Amt.,Deposit Amt.,Closing Balance
40,28/03/25,UPI-NITIN SHARMA-NS290670-1@OKAXIS-UTIB0002471...,0000545364122316,28/03/25,40000,NaN,73029.36
41,28/03/25,UPI-MACFOS LIMITED-MACFOSLIMITED577608.RZP@RXA...,0000545393116668,28/03/25,5401,NaN,67628.36
42,29/03/25,UPI-SOMBARI GOPE-PAYTMQR10DZA2@PAYTM-YESB0PTMU...,0000102227424227,29/03/25,150,NaN,67478.36
43,29/03/25,UPI-BLINKIT-PAYTM-BLINKIT@PTYBL-YESB0PTMUPI-10...,0000102248257143,29/03/25,277,NaN,67201.36
44,01/04/25,INTEREST PAID TILL 31-MAR-2025,000000000000000,31/03/25,NaN,201,67402.36


In [8]:
col_rename_map = {
    "Chq./Ref.No.": "ref_num",
    "Value Dt": "value_date",
    "Deposit Amt.": "deposit",
    "Withdrawal Amt.": "withdraw",
    "Closing Balance": "closing_balance"
}
tailless.rename(columns=col_rename_map, inplace=True)
tailless.columns = [i.lower() for i in tailless.columns]
tailless["value_date"] = pd.to_datetime(tailless["value_date"], format="%d/%m/%y")

In [9]:
tailless.head()

,date,narration,ref_num,value_date,withdraw,deposit,closing_balance
1,01/03/25,UPI-CRIB-CRIB.CF@AXISBANK-UTIB0000100-10434965...,0000104349650605,2025-03-01,16500,NaN,99928.54
2,01/03/25,UPI-INDMONEY-INDMONEYMF@ICICI-ICIC0DC0099-5060...,0000506039051820,2025-03-01,10000,NaN,89928.54
3,01/03/25,UPI-INDMONEY-INDMONEYMF@ICICI-ICIC0DC0099-5060...,0000506039059970,2025-03-01,10000,NaN,79928.54
4,01/03/25,UPI-INDMONEY-INDMONEYMF@ICICI-ICIC0DC0099-5060...,0000506039100266,2025-03-01,6000,NaN,73928.54
5,01/03/25,UPI-INDMONEY-INDMONEYMF@ICICI-ICIC0DC0099-5060...,0000506039035874,2025-03-01,4000,NaN,69928.54


In [10]:
min_date = tailless['value_date'].min()
max_date = tailless['value_date'].max()

min_date, max_date

(Timestamp('2025-03-01 00:00:00'), Timestamp('2025-03-31 00:00:00'))

In [11]:
os.makedirs(destination_file_path, exist_ok=True)

In [12]:
output_file_name = f"transactions_{min_date.date()}_to_{max_date.date()}.csv"
output_file_path = os.path.join(destination_file_path, output_file_name)
output_file_path

'/Users/nitinsharma/playground/finance/transformed/transactions_2025-03-01_to_2025-03-31.csv'

In [13]:
tailless.to_csv(output_file_path, index=False)